In [1]:
import gymnasium as gym
import math
import random
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam

In [2]:
env = gym.make("ALE/Breakout-v5") 
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
FIRE=1

In [3]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

state, info = env.reset()

obs, rwd, termin, trunc, info = env.step(FIRE)

rgb2gray(obs).flatten().shape

def prep(state):
    # print(state.shape)
    return rgb2gray(state).flatten()

## FF

In [4]:
class Net(torch.nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.layers = []
        for d in range(len(dims) - 1):
            self.layers += [Layer(dims[d], dims[d + 1])]

    def predict(self, x):
        goodness_per_label = []
        for label in range(3):  # 10 for mnist
            h = torch.tensor([x.tolist() + [label]])
            goodness = []
            for layer in self.layers:
                h = layer(h)
                goodness += [h.pow(2).mean(1)]
            goodness_per_label += [sum(goodness).unsqueeze(1)]
        goodness_per_label = torch.cat(goodness_per_label, 1)
        return goodness_per_label.argmax(1)

    def train(self, x_pos, x_neg):
        h_pos, h_neg = x_pos, x_neg
        for i, layer in enumerate(self.layers):
            print('training layer', i, '...')
            h_pos, h_neg = layer.train(h_pos, h_neg)
    
    def is_good(self, x):
        goodness = []
        h = x.clone().detach()   # .tolist()
        for layer in self.layers:
            h = layer(h)
            goodness += [h.pow(2).mean(1)]
        return -sum(goodness)

In [5]:
class Layer(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)
        self.relu = torch.nn.ReLU()
        self.opt = Adam(self.parameters(), lr=0.0003)
        self.threshold = 2.0
        self.num_epochs = 1

    def forward(self, x):
        # print(x.shape)
        x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4) # normalization
        # torch.mm -> matrix multiplication
        out = self.relu(torch.mm(x_direction, self.weight.T) + self.bias.unsqueeze(0))

        return out

    def train(self, x_pos, x_neg):
        for i in tqdm(range(self.num_epochs)):
            out_pos = self.forward(x_pos)
            out_neg = self.forward(x_neg)
            g_pos = out_pos.pow(2).mean(1)
            g_neg = out_neg.pow(2).mean(1)
            loss = torch.log(1 + torch.exp(torch.cat([g_pos - self.threshold, -g_neg + self.threshold]))).mean()
            self.opt.zero_grad()
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=2)
            self.opt.step()
        return self.forward(x_pos).detach(), self.forward(x_neg).detach()

In [6]:
from random import sample
from random import random
# from statistics import mean
        
def get_posneg_data(epsilon=0.5, N=5, look_next = False, thresh=0.5):
    negative_data = []
    positive_data = []
    game_lens = []
    for i_episode in range(20):
        state, info = env.reset()
        game = []
        for t in count():
            state_p = prep(state)
            s = random()
            if s > epsilon:
                action = env.action_space.sample()
            else:
                action = net.predict(torch.tensor(state_p)).item()
            observation, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            if terminated:
                next_state = None
                game.append(state_p.tolist() + [action])
            else:
                next_state = observation  
                game.append(state_p.tolist() + [action])
                
                if look_next and (net.is_good(torch.tensor([next_state.tolist() + [0]])) < thresh and
                                  net.is_good(torch.tensor([next_state.tolist() + [1]])) < thresh):
                    negative_data.append(state.tolist() + [action])
            
            state = next_state
            if done:
                break
          
        game_lens.append(len(game))
        if len(game) > N and not truncated:
            negative_data += game[-N:]
            positive_data += game[:-N]
        else:
            negative_data += game
      
    mean_len=sum(game_lens)/20
    print(mean_len)
    pos_len = len(positive_data) 
    neg_len = len(negative_data)
    print(pos_len)
    print(neg_len)
    if pos_len > neg_len:
        positive_data = sample(positive_data, neg_len)
    else:
        negative_data = sample(negative_data, pos_len)
    
    return torch.tensor(positive_data), torch.tensor(negative_data), mean_len

In [7]:
net = Net([210*160+1, 200, 200])
# env = gym.make("CartPole-v1")
env = gym.make("ALE/Breakout-v5") 
n_iters = 50

mean_lens = []
m = 10

for i in range(n_iters):
    eps = 0.8*(1-i/n_iters)
    pos_data, neg_data, m = get_posneg_data(epsilon=eps, N=round(0.3*m)) # , look_next = l_n, thresh=0.2
    mean_lens.append(m)
    net.train(pos_data, neg_data)

264.2
5224
60
training layer 0 ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.79it/s]


training layer 1 ...


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 332.62it/s]


274.7
3914
1580
training layer 0 ...


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 73.90it/s]


177.0
2360
1180
training layer 0 ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.50it/s]


training layer 1 ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 94.93it/s]


In [8]:
print(mean_lens)

[264.2, 274.7, 266.8, 267.25, 251.05, 257.65, 231.25, 274.6, 237.3, 270.15, 254.5, 245.8, 251.35, 246.5, 260.05, 249.25, 241.1, 214.55, 215.4, 230.55, 231.55, 247.05, 235.55, 221.05, 207.2, 211.45, 253.45, 241.2, 207.75, 236.35, 246.35, 221.55, 220.35, 226.05, 212.4, 235.65, 219.4, 217.1, 232.45, 220.15, 214.8, 209.55, 206.0, 241.0, 201.65, 225.4, 188.5, 201.35, 197.05, 177.0]


In [ ]:
import matplotlib.pyplot as plt

# 30 % of game is negative
plt.plot([263.85, 237.0, 224.2, 206.0, 215.5, 202.5, 211.75, 233.5, 212.5, 219.4])
plt.title("mean length of a game during the training")
plt.xlabel("epoch")
plt.ylabel("length")
plt.show()